In [6]:
import cv2
import numpy as np
from google.colab.patches import cv2_imshow
import glob

# Get the list of calibration images
images = glob.glob('/content/drive/MyDrive/calibration_images/calibration_images/*JPEG')

# Chessboard parameters
chessboard_size = (8, 6)
square_size = 0.01

# Lists to store object points and image points
object_points = []  # 3D points in real world space
image_points = []   # 2D points in image plane

# Prepare the chessboard pattern in 3D
object_points_chessboard = np.zeros((chessboard_size[0] * chessboard_size[1], 3), np.float32)
object_points_chessboard[:, :2] = np.mgrid[0:chessboard_size[0], 0:chessboard_size[1]].T.reshape(-1, 2) * square_size

# Iterator variable
image_index = 0

# Loop through images for calibration using a while loop
while image_index < len(images):
    # Read the image
    image = cv2.imread(images[image_index])

    # Convert the image to grayscale
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Find chessboard corners
    ret, corners = cv2.findChessboardCorners(gray_image, chessboard_size, None)

    # If corners are found, add points to the lists
    if ret:
        object_points.append(object_points_chessboard)
        image_points.append(corners)

    # Move to the next image
    image_index += 1

# Get the height and width of the last image
image_height, image_width = image.shape[:2]

# Calibrate the camera
ret, camera_matrix, distortion_coefficients, rotation_vectors, translation_vectors = cv2.calibrateCamera(
    object_points, image_points, (image_height, image_width), None, None
)

# Extract components from the camera matrix
principal_point_x = camera_matrix[1, 2]
principal_point_y = camera_matrix[0, 2]
focal_length_x = camera_matrix[0, 0]
focal_length_y = camera_matrix[1, 1]

# Print results
print(f'Principal Point (cx): {principal_point_x}')
print(f'Principal Point (cy): {principal_point_y}')
print(f'Focal Length (fx): {focal_length_x}')
print(f'Focal Length (fy): {focal_length_y}')


Principal Point (cx): 552.3324481725065
Principal Point (cy): 979.1339630714849
Focal Length (fx): 1476.3602414447728
Focal Length (fy): 1482.1826769617012


In [9]:
# Import required modules
import cv2
import numpy as np
import os
import glob
from google.colab.patches import cv2_imshow


# Define the dimensions of checkerboard
CHECKERBOARD = (6, 8)


# stop the iteration when specified
# accuracy, epsilon, is reached or
# specified number of iterations are completed.
criteria = (cv2.TERM_CRITERIA_EPS +
			cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)


# Vector for 3D points
threedpoints = []

# Vector for 2D points
twodpoints = []


# 3D points real world coordinates
objectp3d = np.zeros((1, CHECKERBOARD[0]
					* CHECKERBOARD[1],
					3), np.float32)
objectp3d[0, :, :2] = np.mgrid[0:CHECKERBOARD[0],
							0:CHECKERBOARD[1]].T.reshape(-1, 2)
prev_img_shape = None


# Extracting path of individual image stored
# in a given directory. Since no path is
# specified, it will take current directory
# jpg files alone
images = glob.glob('/content/drive/MyDrive/calibration_images/calibration_images/*JPEG')

for filename in images:
	image = cv2.imread(filename)
	grayColor = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

	# Find the chess board corners
	# If desired number of corners are
	# found in the image then ret = true
	ret, corners = cv2.findChessboardCorners(
					grayColor, CHECKERBOARD,
					cv2.CALIB_CB_ADAPTIVE_THRESH
					+ cv2.CALIB_CB_FAST_CHECK +
					cv2.CALIB_CB_NORMALIZE_IMAGE)

	# If desired number of corners can be detected then,
	# refine the pixel coordinates and display
	# them on the images of checker board
	if ret == True:
		threedpoints.append(objectp3d)

		# Refining pixel coordinates
		# for given 2d points.
		corners2 = cv2.cornerSubPix(
			grayColor, corners, (11, 11), (-1, -1), criteria)

		twodpoints.append(corners2)

		# Draw and display the corners
		image = cv2.drawChessboardCorners(image,
										CHECKERBOARD,
										corners2, ret)

	cv2_imshow(image)
	# cv2.waitKey(0)

cv2.destroyAllWindows()

h, w = image.shape[:2]


# Perform camera calibration by
# passing the value of above found out 3D points (threedpoints)
# and its corresponding pixel coordinates of the
# detected corners (twodpoints)
ret, matrix, distortion, r_vecs, t_vecs = cv2.calibrateCamera(
	threedpoints, twodpoints, grayColor.shape[::-1], None, None)


# Displaying required output
print(" Camera matrix:")
print(matrix)

print("\n Distortion coefficient:")
print(distortion)

# print("\n Rotation Vectors:")
# print(r_vecs)

# print("\n Translation Vectors:")
# print(t_vecs)





Output hidden; open in https://colab.research.google.com to view.